# Loan Prediction

## 0. Import Packages and Settings

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

## 1. Prepare Data and Make Some Explorations

In [ ]:
loan_train = pd.read_csv("/kaggle/input/loanprediction/train_ctrUa4K.csv")
print("Print the first 5 records in the dataset:")
print(loan_train.head())
print()
print("Basic descriptive statistics of all the variables:")
print(loan_train.describe())
print()
print("Present the feature attributes:")
print(loan_train.info())

In [ ]:
# count the null columns
null_columns = loan_train.columns[loan_train.isnull().any()]
loan_train[null_columns].isnull().sum()

We find that some of the features get missing value, and we need to drop of input some value of them.
We find there are seven features have this situation: Gender, Married, Dependents, Self_Employed, LoanAmount, Loan_Amount_Term, and Credit_History.
We make a quick guess before we check the distribution of the variables:
1. Loan_ID is not relevent.
2. Gender should not be relevent in ideal.
3. Married 'Yes' will have more chance to be approved.
4. Dependents 'With Dependents' will have more chance to get loan, too.
5. Education Higher will be better.
6. Self_Employed not sure.
7. ApplicantIncome Higher will be better.
8. CoapplicantIncome Higher will be better.
9. LoanAmount not sure.
10. Loan_Amount_Term not sure.
11. Credit_History 'With credit history' will be better.
12. Property_Area 'Urban' will be better.

In [ ]:
loan_train.hist(bins=50, figsize=(15, 10))
plt.show()

In [ ]:
df_obj = loan_train.select_dtypes(include='object')

for col in df_obj.iloc[:, 1:].columns:
    print(sns.countplot(x=col, data=df_obj))
    plt.show()

Check the distribution of each variable between Loan_Status = 'Y' v.s. 'N'.

In [ ]:
df_approved = loan_train[loan_train['Loan_Status'] == 'Y']
df_rejected = loan_train[loan_train['Loan_Status'] == 'N']

non_object_variables = ['ApplicantIncome', 'CoapplicantIncome', 
                        'Credit_History', 'LoanAmount', 'Loan_Amount_Term']

for obj in non_object_variables:
    sns.distplot(df_approved[obj][df_approved[obj].isnull() == False], label = 'Loan_Status == Y')
    sns.distplot(df_rejected[obj][df_rejected[obj].isnull() == False], label = 'Loan_Status == N')
    plt.legend()
    plt.show()

In [ ]:
object_variables = ['Gender', 'Married', 'Dependents', 
                   'Self_Employed', 'Property_Area']

for obj in object_variables:
    print(sns.countplot(x=obj, data=df_obj, hue='Loan_Status'))
    plt.show()

We conclude that the higher Y/N ratio, the higher possibility to get loan approved.

## 2. Feature Engineering

### 2.1 Fill the Missing Value with mode() (the value appear the most often)

In [ ]:
def fill_with_mode(df, x):
    df[x].fillna(df[x].mode()[0], inplace=True)
    
has_null_objects = ['Gender', 'Married', 'Dependents',
                    'Self_Employed', 'LoanAmount', 'Loan_Amount_Term',
                    'Credit_History']

for obj in has_null_objects:
    fill_with_mode(loan_train, obj)

In [ ]:
# make sure all null value has bee filled
for col in loan_train.columns:
    print('Variable', col, 'has missing entry:', sum(loan_train[col].isnull()))

In [ ]:
X = loan_train.iloc[:, 1:-1]
y = loan_train.iloc[:, -1]

In [ ]:
print(X.shape)
print(y.shape)

### 2.2 One Hot Encoding to Categorical Features

In [ ]:
# these four columns need to be onehotted
onehot_targets = ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 
                  'Property_Area']

In [ ]:
for target in onehot_targets:
    onehot_temp = pd.get_dummies(X[target])
    X = X.drop(target, axis=1)
    X = pd.concat([onehot_temp, X], axis=1)

### 2.3 Label Encoding to Predicted Value

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y)

In [ ]:
y = le.transform(y)

In [ ]:
# prepare train and validation data
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

## 3. Prepare Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
clf = RandomForestClassifier(n_estimators=10, max_depth=None, min_samples_split=2, random_state=0)
scores = cross_val_score(clf, X, y, cv=5)
print("Random forest Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

## 4. Hyperparameter Searching with RandomSearchCV

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start=10, stop=700, num=10)]
# number of features to consider in every split
max_features = ['auto', 'sqrt']
# maximum level in tree
max_depth = [int(x) for x in np.linspace(10, 110, num=11)]
max_depth.append(None)
# minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# method of selecting samples of training each tree
bootstrap = [True, False]

# create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

In [ ]:
# start hyperparameter searching
rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid, n_iter=100,
                              cv=5, verbose=2, random_state=42, n_jobs=-1)
rf_random.fit(X, y)

In [ ]:
# print the best hyperparameters
rf_random.best_params_

## 5. Train and Evaulate Model

In [ ]:
clrf_grid = RandomForestClassifier(n_estimators=546, min_samples_split=2, min_samples_leaf=4,
                                  max_features='sqrt', max_depth=10, bootstrap=True)
scores = cross_val_score(clrf_grid, X, y, cv=5)

In [ ]:
print("Grid Random Forst Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

## 6. Work on Test Data

In [ ]:
loan_test = pd.read_csv('/kaggle/input/loanprediction/test_lAUu6dG.csv')
    
X_test = loan_test.iloc[:, 1:]
    
for feature in has_null_objects:
    fill_with_mode(X_test, feature)
    
# make sure all null value has bee filled
for col in X_test.columns:
    print('Variable', col, 'has missing entry:', sum(X_test[col].isnull()))

In [ ]:
# onehot encoding to test data specific features
for target in onehot_targets:
    onehot_temp = pd.get_dummies(X_test[target])
    X_test = X_test.drop(target, axis=1)
    X_test = pd.concat([onehot_temp, X_test], axis=1)

In [ ]:
clrf_grid.fit(X, y)

results = clrf_grid.predict(X_test)
SJ_submit = pd.DataFrame({"Loan_ID": loan_test['Loan_ID'], "Loan_Status": results})
print(SJ_submit.head())

In [ ]:
pd.DataFrame(SJ_submit).to_csv("submit_CudaChen.csv", index=False)